<a href="https://colab.research.google.com/github/bonguchandu123/HTMLPROJ/blob/main/htmlfraud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi uvicorn pyngrok scikit-learn pandas pennylane matplotlib



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 102.8 MB/s eta 0:00:00


In [ ]:
!pip install uvicorn

In [ ]:
!ngrok config add-authtoken 31e3W5Dm2p41ltYXjyGN5qJMUOc_3Pj1mN7dntp2JXHNceREo


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [11]:
from fastapi import FastAPI, File, UploadFile, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import numpy as np
import pandas as pd
import io
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import pennylane as qml
from pennylane import numpy as qnp
import pickle
import json
from typing import List, Optional
import time
import asyncio
import nest_asyncio
import uvicorn
from pyngrok import ngrok


app = FastAPI(title="Quantum Fraud Detection API", version="1.0.0")

# CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Global variables to store models
quantum_model = None
scaler_quantum = None
classical_models = {}
scaler_classical = None

# Quantum setup
n_qubits = 4
dev = qml.device("default.qubit", wires=n_qubits)

def feature_map(x):
    for i in range(n_qubits):
        qml.RY(x[i], wires=i)
    for i in range(n_qubits-1):
        qml.CNOT(wires=[i, i+1])

def ansatz(weights):
    for i in range(n_qubits):
        qml.RY(weights[i], wires=i)
        qml.RZ(weights[i+n_qubits], wires=i)

@qml.qnode(dev, interface="autograd")
def circuit(x, weights):
    feature_map(x)
    ansatz(weights)
    return qml.expval(qml.PauliZ(0))

def predict_batch(W, Xb):
    outs = []
    for x in Xb:
        z = circuit(x, W)
        p = 0.5 * (1 + z)
        outs.append(qnp.clip(p, 1e-7, 1-1e-7))
    return qnp.stack(outs)

def bce_loss(W, Xb, yb):
    p = predict_batch(W, Xb)
    return -qnp.mean(yb*qnp.log(p) + (1-yb)*qnp.log(1-p))

def make_toy_fraud(n=1200, seed=42):
    rng = np.random.default_rng(seed)
    n0 = int(n*0.9)
    X0 = rng.normal(loc=[0.2,0.4,0.3,0.2], scale=[0.1,0.15,0.15,0.1], size=(n0,4))
    y0 = np.zeros(n0)
    n1 = n - n0
    X1 = rng.normal(loc=[0.7,0.8,0.8,0.7], scale=[0.15,0.15,0.1,0.1], size=(n1,4))
    y1 = np.ones(n1)
    X = np.vstack([X0, X1])
    y = np.concatenate([y0, y1])
    X = np.clip(X, 0, 1)
    return X, y.astype(int)

# Pydantic models
class TransactionInput(BaseModel):
    amount: float
    hour: int
    device: str
    merchant_risk: float
    merchant_category: str
    transaction_type: str
    cardholder_age: int

class TrainingConfig(BaseModel):
    epochs: int = 20
    batch_size: int = 64
    stepsize: float = 0.2
    seed: int = 123

class PredictionResponse(BaseModel):
    quantum_prediction: float
    classical_rf_prediction: float
    classical_lr_prediction: float
    hybrid_prediction: float
    is_fraud: bool

class TrainingResponse(BaseModel):
    success: bool
    message: str
    metrics: dict

class AnalyticsResponse(BaseModel):
    model_performance: dict
    feature_importance: dict
    confusion_matrix: List[List[int]]

# API Routes
@app.get("/")
async def root():
    return {"message": "Quantum Fraud Detection API is running"}

@app.get("/health")
async def health_check():
    return {"status": "healthy", "quantum_device": str(dev)}

@app.post("/upload-csv")
async def upload_csv(file: UploadFile = File(...)):
    try:
        content = await file.read()
        df = pd.read_csv(io.StringIO(content.decode('utf-8')))

        # Validate required columns
        required_cols = ["amount", "time", "device", "merchant_risk", "label",
                        "merchant_category", "transaction_type", "cardholder_age"]
        missing_cols = [col for col in required_cols if col.lower() not in [c.lower() for c in df.columns]]

        if missing_cols:
            raise HTTPException(status_code=400, detail=f"Missing columns: {missing_cols}")

        return {
            "success": True,
            "message": f"CSV uploaded successfully with {len(df)} rows",
            "preview": df.head().to_dict('records'),
            "columns": df.columns.tolist(),
            "shape": df.shape
        }
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

@app.post("/train-quantum", response_model=TrainingResponse)
async def train_quantum_model(config: TrainingConfig):
    global quantum_model, scaler_quantum

    try:
        # Generate toy data for training
        X, y = make_toy_fraud(n=1400, seed=config.seed)

        # Scale features
        scaler_quantum = StandardScaler()
        X_scaled = scaler_quantum.fit_transform(X)

        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X_scaled, y, test_size=0.2, random_state=config.seed, stratify=y
        )

        # Initialize quantum weights
        qnp.random.seed(config.seed)
        W = qnp.array(0.01 * qnp.random.randn(2*n_qubits), requires_grad=True)

        # Training
        opt = qml.GradientDescentOptimizer(stepsize=config.stepsize)
        X_train_q = qnp.array(X_train, requires_grad=False)
        y_train_q = qnp.array(y_train, requires_grad=False)

        for epoch in range(config.epochs):
            idx = np.random.choice(len(X_train), min(config.batch_size, len(X_train)), replace=False)
            batchX = X_train_q[idx]
            batchY = y_train_q[idx]
            W = opt.step(lambda w: bce_loss(w, batchX, batchY), W)

        # Evaluation
        probs = predict_batch(W, qnp.array(X_test))
        preds = (probs > 0.5).astype(int)
        auc = roc_auc_score(y_test, np.asarray(probs))
        f1 = f1_score(y_test, np.asarray(preds))

        # Store model
        quantum_model = W

        metrics = {
            "auc": float(auc),
            "f1": float(f1),
            "test_accuracy": float(np.mean(preds == y_test))
        }

        return TrainingResponse(
            success=True,
            message="Quantum model trained successfully",
            metrics=metrics
        )

    except Exception as e:
        return TrainingResponse(
            success=False,
            message=f"Training failed: {str(e)}",
            metrics={}
        )

@app.post("/train-classical", response_model=TrainingResponse)
async def train_classical_models():
    global classical_models, scaler_classical

    try:
        # Generate toy data
        X, y = make_toy_fraud(n=1400, seed=42)
        X_extra = np.zeros((len(X), 3))  # Mock extra features
        X_all = np.hstack([X, X_extra])

        # Scale features
        scaler_classical = StandardScaler()
        X_scaled = scaler_classical.fit_transform(X_all)

        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X_scaled, y, test_size=0.2, random_state=42, stratify=y
        )

        # Train models
        rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
        lr_model = LogisticRegression(random_state=42)

        rf_model.fit(X_train, y_train)
        lr_model.fit(X_train, y_train)

        # Evaluate
        rf_probs = rf_model.predict_proba(X_test)[:, 1]
        lr_probs = lr_model.predict_proba(X_test)[:, 1]

        rf_auc = roc_auc_score(y_test, rf_probs)
        lr_auc = roc_auc_score(y_test, lr_probs)

        # Store models
        classical_models = {
            "random_forest": rf_model,
            "logistic_regression": lr_model
        }

        metrics = {
            "random_forest_auc": float(rf_auc),
            "logistic_regression_auc": float(lr_auc)
        }

        return TrainingResponse(
            success=True,
            message="Classical models trained successfully",
            metrics=metrics
        )

    except Exception as e:
        return TrainingResponse(
            success=False,
            message=f"Training failed: {str(e)}",
            metrics={}
        )

@app.post("/predict", response_model=PredictionResponse)
async def predict_transaction(transaction: TransactionInput):
    global quantum_model, scaler_quantum, classical_models, scaler_classical

    if quantum_model is None or not classical_models:
        raise HTTPException(status_code=400, detail="Models not trained. Please train models first.")

    try:
        # Map categorical features
        device_map = {"Mobile": 0.2, "Desktop": 0.5, "ATM": 0.8}
        cat_map = {"Electronics": 0.0, "Grocery": 0.5, "Entertainment": 1.0}
        type_map = {"Online": 0.2, "In-Person": 0.5, "ATM": 0.8}

        # Prepare quantum features
        x_quantum = np.array([
            transaction.amount / 1000.0,
            transaction.hour / 24.0,
            device_map.get(transaction.device, 0.5),
            transaction.merchant_risk
        ])

        # Prepare classical features
        x_extra = np.array([
            cat_map.get(transaction.merchant_category, 0.5),
            type_map.get(transaction.transaction_type, 0.5),
            transaction.cardholder_age / 100.0
        ])

        x_classical = np.hstack([x_quantum, x_extra])

        # Scale features
        x_quantum_scaled = scaler_quantum.transform([x_quantum])
        x_classical_scaled = scaler_classical.transform([x_classical])

        # Quantum prediction
        quantum_prob = float(predict_batch(quantum_model, qnp.array(x_quantum_scaled))[0])

        # Classical predictions
        rf_prob = float(classical_models["random_forest"].predict_proba(x_classical_scaled)[:, 1][0])
        lr_prob = float(classical_models["logistic_regression"].predict_proba(x_classical_scaled)[:, 1][0])

        # Hybrid prediction
        hybrid_prob = (quantum_prob + rf_prob) / 2

        return PredictionResponse(
            quantum_prediction=quantum_prob,
            classical_rf_prediction=rf_prob,
            classical_lr_prediction=lr_prob,
            hybrid_prediction=hybrid_prob,
            is_fraud=hybrid_prob > 0.5
        )

    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

@app.get("/analytics", response_model=AnalyticsResponse)
async def get_analytics():
    global quantum_model, classical_models, scaler_quantum, scaler_classical

    if quantum_model is None or not classical_models:
        raise HTTPException(status_code=400, detail="Models not trained. Please train models first.")

    try:
        # Generate test data for analytics
        X, y = make_toy_fraud(n=500, seed=999)
        X_extra = np.zeros((len(X), 3))
        X_all = np.hstack([X, X_extra])

        # Scale features
        X_quantum_scaled = scaler_quantum.transform(X)
        X_classical_scaled = scaler_classical.transform(X_all)

        # Get predictions
        quantum_probs = np.array([float(p) for p in predict_batch(quantum_model, qnp.array(X_quantum_scaled))])
        rf_probs = classical_models["random_forest"].predict_proba(X_classical_scaled)[:, 1]
        lr_probs = classical_models["logistic_regression"].predict_proba(X_classical_scaled)[:, 1]

        # Calculate metrics
        quantum_auc = roc_auc_score(y, quantum_probs)
        rf_auc = roc_auc_score(y, rf_probs)
        lr_auc = roc_auc_score(y, lr_probs)

        # Feature importance (mock for quantum)
        feature_names = ["Amount", "Time", "Device", "Merchant Risk"]
        rf_importance = classical_models["random_forest"].feature_importances_[:4]  # First 4 features

        # Confusion matrix for quantum model
        quantum_preds = (quantum_probs > 0.5).astype(int)
        cm = confusion_matrix(y, quantum_preds).tolist()

        return AnalyticsResponse(
            model_performance={
                "quantum_auc": float(quantum_auc),
                "random_forest_auc": float(rf_auc),
                "logistic_regression_auc": float(lr_auc)
            },
            feature_importance={
                feature_names[i]: float(rf_importance[i]) for i in range(len(feature_names))
            },
            confusion_matrix=cm
        )

    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

@app.post("/simulate-transactions")
async def simulate_random_transactions(count: int = 10):
    global quantum_model, classical_models, scaler_quantum, scaler_classical

    if quantum_model is None or not classical_models:
        raise HTTPException(status_code=400, detail="Models not trained. Please train models first.")

    try:
        results = []
        device_map = {"Mobile": 0.2, "Desktop": 0.5, "ATM": 0.8}
        cat_map = {"Electronics": 0.0, "Grocery": 0.5, "Entertainment": 1.0}
        type_map = {"Online": 0.2, "In-Person": 0.5, "ATM": 0.8}

        for i in range(count):
            # Generate random transaction
            amount = np.random.randint(10, 2000)
            hour = np.random.randint(0, 24)
            device = np.random.choice(["Mobile", "Desktop", "ATM"])
            merchant_risk = np.random.rand()
            category = np.random.choice(["Electronics", "Grocery", "Entertainment"])
            trans_type = np.random.choice(["Online", "In-Person", "ATM"])
            age = np.random.randint(18, 80)

            # Prepare features
            x_quantum = np.array([
                amount / 1000.0, hour / 24.0,
                device_map[device], merchant_risk
            ])
            x_extra = np.array([
                cat_map[category], type_map[trans_type], age / 100.0
            ])
            x_classical = np.hstack([x_quantum, x_extra])

            # Scale and predict
            x_quantum_scaled = scaler_quantum.transform([x_quantum])
            x_classical_scaled = scaler_classical.transform([x_classical])

            quantum_prob = float(predict_batch(quantum_model, qnp.array(x_quantum_scaled))[0])
            rf_prob = float(classical_models["random_forest"].predict_proba(x_classical_scaled)[:, 1][0])
            hybrid_prob = (quantum_prob + rf_prob) / 2

            results.append({
                "id": i + 1,
                "amount": amount,
                "hour": hour,
                "device": device,
                "merchant_risk": round(merchant_risk, 3),
                "category": category,
                "transaction_type": trans_type,
                "age": age,
                "quantum_prediction": round(quantum_prob, 3),
                "rf_prediction": round(rf_prob, 3),
                "hybrid_prediction": round(hybrid_prob, 3),
                "is_fraud": hybrid_prob > 0.5
            })

            # Small delay for realistic simulation
            await asyncio.sleep(0.1)

        return {"transactions": results}

    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

@app.get("/model-status")
async def get_model_status():
    return {
        "quantum_model_trained": quantum_model is not None,
        "classical_models_trained": bool(classical_models),
        "available_models": list(classical_models.keys()) if classical_models else []
    }


# =========================
# Start FastAPI + ngrok in Colab
# =========================
if __name__ == "__main__":
    nest_asyncio.apply()

    # Start ngrok tunnel
    public_url = ngrok.connect(8000)
    print("🚀 FastAPI is live at:", public_url)

    # Allow CORS for ngrok URL
    app.add_middleware(
        CORSMiddleware,
        allow_origins=[str(public_url), "http://localhost:3000", "http://localhost:5173"],
        allow_credentials=True,
        allow_methods=["*"],
        allow_headers=["*"],
    )

    uvicorn.run(app, host="0.0.0.0", port=8000)


🚀 FastAPI is live at: NgrokTunnel: "https://6743167504cc.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [147]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     2409:40f0:410c:41ea:442b:1ba1:c69e:ce19:0 - "OPTIONS /model-status HTTP/1.1" 200 OK
INFO:     2409:40f0:410c:41ea:442b:1ba1:c69e:ce19:0 - "OPTIONS /model-status HTTP/1.1" 200 OK
INFO:     2409:40f0:410c:41ea:442b:1ba1:c69e:ce19:0 - "GET /model-status HTTP/1.1" 200 OK
INFO:     2409:40f0:410c:41ea:442b:1ba1:c69e:ce19:0 - "GET /model-status HTTP/1.1" 200 OK
INFO:     2409:40f0:410c:41ea:442b:1ba1:c69e:ce19:0 - "OPTIONS /health HTTP/1.1" 200 OK
INFO:     2409:40f0:410c:41ea:442b:1ba1:c69e:ce19:0 - "GET /health HTTP/1.1" 200 OK
INFO:     2409:40f0:410c:41ea:442b:1ba1:c69e:ce19:0 - "OPTIONS /train-quantum HTTP/1.1" 200 OK


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-195' coro=<Server.serve() done, defined at /usr/local/lib/python3.12/dist-packages/uvicorn/server.py:69> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/server.py", line 67, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.12/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.12/asyncio/events.py", line 88, in _run
    

INFO:     2409:40f0:410c:41ea:442b:1ba1:c69e:ce19:0 - "POST /train-quantum HTTP/1.1" 200 OK
INFO:     2409:40f0:410c:41ea:442b:1ba1:c69e:ce19:0 - "OPTIONS /train-classical HTTP/1.1" 200 OK
INFO:     2409:40f0:410c:41ea:442b:1ba1:c69e:ce19:0 - "GET /model-status HTTP/1.1" 200 OK
INFO:     2409:40f0:410c:41ea:442b:1ba1:c69e:ce19:0 - "POST /train-classical HTTP/1.1" 200 OK
INFO:     2409:40f0:410c:41ea:442b:1ba1:c69e:ce19:0 - "GET /model-status HTTP/1.1" 200 OK
INFO:     2409:40f0:410c:41ea:442b:1ba1:c69e:ce19:0 - "GET /model-status HTTP/1.1" 200 OK
INFO:     2409:40f0:410c:41ea:442b:1ba1:c69e:ce19:0 - "OPTIONS /simulate-transactions?count=10 HTTP/1.1" 200 OK
INFO:     2409:40f0:410c:41ea:442b:1ba1:c69e:ce19:0 - "POST /simulate-transactions?count=10 HTTP/1.1" 200 OK
INFO:     2409:40f0:410c:41ea:442b:1ba1:c69e:ce19:0 - "OPTIONS /analytics HTTP/1.1" 200 OK
INFO:     2409:40f0:410c:41ea:442b:1ba1:c69e:ce19:0 - "OPTIONS /analytics HTTP/1.1" 200 OK
INFO:     2409:40f0:410c:41ea:442b:1ba1:c69e

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [147]


In [ ]:
!python app.py


Traceback (most recent call last):
  File "/content/app.py", line 35, in <module>
    clf.fit(X, y)
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/ensemble/_forest.py", line 360, in fit
    X, y = validate_data(
           ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py", line 2961, in validate_data
    X, y = check_X_y(X, y, **check_params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py", line 1387, in check_X_y
    y = _check_y(y, multi_output=multi_output, y_numeric=y_numeric, estimator=estimator)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py", line 1397, in _